In [58]:
import numpy as np
import pandas as pd
from osgeo import gdal, osr
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
from scipy.ndimage import gaussian_filter

In [59]:
def CreateGeoTiff(Name, Array, driver, NDV, 
                  GeoT, Projection, DataType):
    Array[np.isnan(Array)] = NDV
    # DataSet = driver.Create(Name, Array.shape[2], Array.shape[1], Array.shape[0], DataType)
    # DataSet = driver.Create(Name, Array.shape[1], Array.shape[0], Array.shape[2], DataType)  ## col, row, bands
    DataSet = driver.Create(Name, Array.shape[2], Array.shape[1], Array.shape[0], DataType)  ## col, row, bands
    DataSet.SetGeoTransform(GeoT)
    DataSet.SetProjection( Projection.ExportToWkt() )
    # for i in range(Array.shape[2]):
    for i in range(Array.shape[0]):       
        # DataSet.GetRasterBand(i+1).WriteArray(Array[:,:,i] )
        DataSet.GetRasterBand(i+1).WriteArray(Array[i,:,:] )
        DataSet.GetRasterBand(i+1).SetNoDataValue(NDV)
    DataSet.FlushCache()
    return Name

In [60]:
### source: https://stackoverflow.com/questions/18697532/gaussian-filtering-a-image-with-nan-in-python

def filter_nan_gaussian_conserving(arr, sigma):
    """Apply a gaussian filter to an array with nans.

    Intensity is only shifted between not-nan pixels and is hence conserved.
    The intensity redistribution with respect to each single point
    is done by the weights of available pixels according
    to a gaussian distribution.
    All nans in arr, stay nans in gauss.
    """
    nan_msk = np.isnan(arr)

    loss = np.zeros(arr.shape)
    loss[nan_msk] = 1
    loss = gaussian_filter(
            loss, sigma=sigma, mode='constant', cval=1)

    gauss = arr.copy()
    gauss[nan_msk] = 0
    gauss = gaussian_filter(
            gauss, sigma=sigma, mode='constant', cval=0)
    gauss[nan_msk] = np.nan

    gauss += loss * arr

    return gauss

In [61]:
# apply msi psf to wv3 VNIR image
# we assume the WV3 bands and corresponding MSI MTFs are as below
# S2 band          MSI MTF stdev
# 1 = coastal       0.0173
# 2 = blue          0.0318
# 3 = green         0.0313
# 4 = red           0.0305
# 5 = red edge1         0.0173
# 6 = red edge2        0.0166
# 7 = red edge3      0.0168
# 8 = nir1          0.0292
# 9 = nri2          0.0163
#10 =SWIR1           0.0137
#11= SWIR2         0.0148
# The MSI MTFs are taken from Li et al. https://www.mdpi.com/2072-4292/12/15/2406/htm
sigmaf = np.array([0.0173,0.0318,0.0313,0.0305,0.0173,0.0166, 0.0168, 0.0292,0.0163,0.0137, 0.0148])
sigmal =  1/(sigmaf*2*3.14159256)
print(sigmal)
numnS2bands = 11;

[ 9.19970797  5.00487257  5.08482261  5.21819501  9.19970797  9.58764746
  9.4735088   5.45051191  9.76410723 11.61714948 10.75371269]


In [62]:
NEON_dir='F:\\neon\\valid\\image\\onemeter\\11bands'

In [63]:
for file in os.listdir(NEON_dir):
    if file.endswith('11bands.tif'):       
        full_path=NEON_dir+'\\'+file
        
        ds_S2 = gdal.Open(full_path)
        ## projection information
        prj=ds_S2.GetProjection()     
        srs=osr.SpatialReference(wkt=prj)
     
        geotransform = ds_S2.GetGeoTransform()
        bands=ds_S2.RasterCount 
        width=ds_S2.RasterXSize 
        height=ds_S2.RasterYSize 
        print (bands, width, height)
        S2_B= np.empty([bands, height, width], dtype=float)
        S2_B = np.array(ds_S2.ReadAsArray())
        # print (S2_B.shape)
        S2_B[S2_B==-9999.]=np.nan
        
        file_name=file.rsplit('.', maxsplit=1)[0]
        file_filter=NEON_dir+'\\'+file_name+'_filt.tif'
        
        for x in range(0,numnS2bands):     
            S2_B[x,:,:] = filter_nan_gaussian_conserving(S2_B[x,:,:],sigmal[x])
        
        data_type = gdal.GDT_Float32
        driver = gdal.GetDriverByName('GTiff')
        CreateGeoTiff(file_filter, S2_B, driver, -9999, geotransform, srs, data_type)
        
        file_filter_10m=NEON_dir+'\\'+file_name+'_filt_10m.tif'
        file_filter_20m=NEON_dir+'\\'+file_name+'_filt_20m.tif'
        
        raster_rprj = gdal.Warp(file_filter_10m, file_filter, xRes=10, yRes=10, resampleAlg = "average")
        raster_rprj = gdal.Warp(file_filter_20m, file_filter, xRes=20, yRes=20, resampleAlg = "average")
        raster_rprj = None

11 1104 6519
11 981 10823
11 1165 13706
11 1420 15392
11 973 6996
11 924 12034
11 995 10856
11 1081 18702
11 1030 11414
11 1141 11602
